In [2]:
import sys
sys.path.append('..')
import feature_spaces
import encoding_utils
import dvu
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import seaborn as sns
import os
import pandas as pd
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeCV, LogisticRegressionCV

from typing import List
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import datasets
import encoding_utils, feature_spaces, imodelsx
%load_ext autoreload
%autoreload 2

from feature_spaces import *
from feature_spaces import em_data_dir, data_dir, results_dir
# from fit_linear_models import get_dsets
# from ridge_utils.SemanticModel import SemanticModel
dvu.set_style()

### Load story texts

In [3]:
# Look at narrative stories
train_stories, test_stories, allstories = encoding_utils.get_allstories([1, 2, 3, 4, 5])
wordseqs = feature_spaces.get_story_wordseqs(train_stories)
texts = [' '.join(wordseqs[k].data) for k in wordseqs.keys()]

In [8]:
with open('narrative_stories.txt', 'w') as f:
    f.write('\n'.join(texts))